In [ ]:
import itertools
import string

def generate_combinations(limit):
    alphabets = list(string.ascii_lowercase[:limit])
    combinations = []
    for r in range(1, limit + 1):
        combinations.extend(itertools.product(alphabets, repeat=r))

    # Print all combinations
    for combo in combinations:
        print("".join(combo))

    print("Total combinations:", len(combinations))

generate_combinations(limit=3)

a
b
c
aa
ab
ac
ba
bb
bc
ca
cb
cc
aaa
aab
aac
aba
abb
abc
aca
acb
acc
baa
bab
bac
bba
bbb
bbc
bca
bcb
bcc
caa
cab
cac
cba
cbb
cbc
cca
ccb
ccc
Total combinations: 39


In [ ]:
import itertools
import string
import corpus

# Function to remove double letters
def __remove_double_letters(word):
    return "".join(ch for i, ch in enumerate(word) if i == 0 or ch != word[i - 1])

# Function to simplify transliteration
def simplify_transliteration(word):
    exclude_word = {'h', 'H', 'w', 'W'}
    exclude_cons = {'a', 'A', 'e', 'E', 'i', 'I', 'o', 'O', 'u', 'U', 'h', 'H', 'w', 'W', 'y', 'Y'}

    word = __remove_double_letters(word)

    simplified_word = ''
    simplified_cons = ''

    for index, char in enumerate(word):
        if char.lower() == 'v':
            char = 'b'
        elif char.lower() == 'f':
            char = 'p'
        elif char.lower() == 'y':
            char = 'i'
        elif char.lower() == 'a':
            char = 'e'
        elif char.lower() in {'s', 'k'}:
            char = 'c'
        elif char.lower() == 'u':
            char = 'o'
        elif char.lower() in {'g', 'z'}:
            char = 'j'

        if index == 0:
            simplified_word += char
            simplified_cons += char
        else:
            if char not in exclude_word:
                simplified_word += char

            if char not in exclude_cons:
                simplified_cons += char

    return simplified_word, simplified_cons

# Function to calculate Levenshtein distance
def levenshtein_distance(s1, s2):
    rows, cols = len(s1) + 1, len(s2) + 1
    dp = [[0] * cols for _ in range(rows)]

    for i in range(rows):
        dp[i][0] = i
    for j in range(cols):
        dp[0][j] = j

    for i in range(1, rows):
        for j in range(1, cols):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[-1][-1]

# Function to find nearest words
# def __find_nearest_words(word, n=3, lang="english", include_same_distance_words=False):
def __find_nearest_words(word, n, lang, include_same_distance_words=False):

    word = word.lower()
    word_sound, word_cons = simplify_transliteration(word)

    nearest_words = []
    unique_words = []
    sound_distance_weight = 0.2
    cons_distance_weight = 0.6
    edit_distance_weight = 0.2

    word_length = len(word)
    ld = word_length
    wl = len(word_cons) * 2
    for dict_word, detail in corpus.corpus.items():

    # for dict_word, detail in corpus.items():
        if ('s' in word and 'k' in dict_word) or ('k' in word and 's' in dict_word):
            continue

        if levenshtein_distance(word, dict_word) > ld:
            continue

        if abs(len(word) - len(dict_word)) > wl:
            continue  # Skip words with large length differences

        dict_sound, dict_cons = simplify_transliteration(dict_word)

        sound_distance = levenshtein_distance(word_sound, dict_sound)
        cons_distance = levenshtein_distance(word_cons, dict_cons)
        edit_distance = levenshtein_distance(word, dict_word)

        combined_distance = (
            (sound_distance_weight * sound_distance) +
            (cons_distance_weight * cons_distance) +
            (edit_distance_weight * edit_distance)
        )

        if lang == 'english':
            nearest_words.append((dict_word, combined_distance))
        else:
            nearest_words.append((dict_word, combined_distance, detail['bangla'][0]))

    nearest_words.sort(key=lambda x: x[1])  # Sort by distance

    current_distance = 0
    for nword, distance in nearest_words:
        if nword not in unique_words:
            if len(unique_words) < n:
                unique_words.append(nword)
                current_distance = distance
            elif include_same_distance_words and current_distance == distance:
                unique_words.append(nword)
            else:
                break

    return unique_words

# Function to generate all letter combinations and find nearest words
def generate_combinations(lower_limit, upper_limit):
    alphabets = list(string.ascii_lowercase[:upper_limit])
    combinations = []

    for r in range(lower_limit, upper_limit + 1):
        combinations.extend(itertools.product(alphabets, repeat=r))

    for combo in combinations:
        word = "".join(combo)
        nearest_words = __find_nearest_words(word, 5, "english")
        nearest_words_bn = __find_nearest_words(word, 5, "english")

        print(f"{word} -> en {nearest_words}, bn {nearest_words_bn}")

    print("Total combinations:", len(combinations))

# Run the function
generate_combinations(lower_limit=2, upper_limit=3)

In [ ]:
#Final Draft 2 for corpus format
import csv
import json
import unicodedata

def soundex(query: str):
    query = query.lower().strip()
    letters = [char for char in query if char.isalpha()]

    if len(letters) == 0:
        return "0000"

    first_letter = letters[0]
    letters = letters[1:]
    letters = [char for char in letters if char not in ('a', 'e', 'i', 'o', 'u', 'y', 'h', 'w')]

    if len(letters) == 0:
        return first_letter.upper() + "000"

    to_replace = {
        ('b', 'f', 'p', 'v'): 1,
        ('c', 'g', 'j', 'k', 'q', 's', 'x', 'z'): 2,
        ('d', 't'): 3,
        ('l',): 4,
        ('m', 'n'): 5,
        ('r',): 6
    }

    first_letter_digit = next((value for group, value in to_replace.items() if first_letter in group), first_letter)
    letters = [next((value for group, value in to_replace.items() if char in group), char) for char in letters]
    letters = [char for i, char in enumerate(letters) if i == 0 or char != letters[i - 1]]

    if isinstance(first_letter_digit, int) and first_letter_digit == letters[0]:
        letters[0] = query[0]
    else:
        letters.insert(0, first_letter)

    letters = [char for char in letters if isinstance(char, int)][:3]
    while len(letters) < 3:
        letters.append(0)

    return str(first_letter).upper() + ''.join(map(str, letters))


def csv_to_dict(csv_files):
    DICTIONARY = {}
    sl = 0

    for csv_file in csv_files:
        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)

            for row in reader:
                if row.get('Name?', '').strip().lower() == 'y':
                    continue
                key = row['Corrected'].strip() if row['Corrected'].strip() else row['English'].strip().lower()
                sl += 1
                bangla_value = unicodedata.normalize('NFC', row['Bangla'].strip())

                try:
                    soundex_code = soundex(key)
                except IndexError as e:
                    print(f"Error processing key '{key}' at entry {sl}: {e}")
                    continue

                if key in DICTIONARY:
                    normalized_existing = [unicodedata.normalize('NFC', val) for val in DICTIONARY[key]['bangla']]
                    if bangla_value not in normalized_existing:
                        DICTIONARY[key]['bangla'].append(bangla_value)
                else:
                    DICTIONARY[key] = {
                        'bangla': [bangla_value]
                    }

    return DICTIONARY


csv_files = [
    '/content/bangla_scraped_words_v2 - 10k_1.csv',
    '/content/bangla_scraped_words_v2 - 10k_2.csv',
    '/content/bangla_scraped_words_v2 - 10k_3.csv',
    '/content/bangla_scraped_words_v2 - 10k_4.csv',
    '/content/bangla_scraped_words_v2 - 10k_5.csv',
    '/content/bangla_scraped_words_v2 - 10k_6.csv',
    '/content/bangla_scraped_words_v2 - 10k_7.csv',
    '/content/bangla_scraped_words_v2 - 10k_8.csv',
    '/content/bangla_scraped_words_v2 - 10k_9.csv',
    '/content/bangla_scraped_words_v2 - 10k_10.csv',
    '/content/bangla_scraped_words_v2 - old_10k_1.csv'
]

combined_dictionary_output = csv_to_dict(csv_files)

print(json.dumps(combined_dictionary_output, indent=4, ensure_ascii=False))

file_path = "/content/corpus.py"
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(f"dictionary={json.dumps(combined_dictionary_output, ensure_ascii=False, separators=(',', ':'))}")


In [ ]:
# corpus_merged.json
import csv
import json

csv_files = [
    '/content/bangla_scraped_words_v2 - 10k_1.csv',
    '/content/bangla_scraped_words_v2 - 10k_2.csv',
    '/content/bangla_scraped_words_v2 - 10k_3.csv',
    '/content/bangla_scraped_words_v2 - 10k_4.csv',
    '/content/bangla_scraped_words_v2 - 10k_5.csv',
    '/content/bangla_scraped_words_v2 - 10k_6.csv',
    '/content/bangla_scraped_words_v2 - 10k_7.csv',
    '/content/bangla_scraped_words_v2 - 10k_8.csv',
    '/content/bangla_scraped_words_v2 - 10k_9.csv',
    '/content/bangla_scraped_words_v2 - 10k_10.csv',
    '/content/bangla_scraped_words_v2 - old_10k_1.csv'
]

# Final corpus dictionary: { "key_word": "bangla_word1_bangla_word2" }
corpus_merged = {}

for csv_file in csv_files:
    try:
        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row.get('Name?', '').strip().lower() == 'y':
                    continue

                key = row.get('Corrected', '').strip()
                if not key:
                    key = row.get('English', '').strip().lower()

                bangla = row.get('Bangla', '').strip()

                if key and bangla:
                    if key not in corpus_merged:
                        corpus_merged[key] = bangla
                    else:
                        existing = corpus_merged[key].split('_')
                        if bangla not in existing:
                            corpus_merged[key] += f'_{bangla}'

    except Exception as e:
        print(f"Failed to process file {csv_file}: {e}")

corpus_merged_path = '/content/corpus_merged.json'
with open(corpus_merged_path, 'w', encoding='utf-8') as json_file:
    json.dump(corpus_merged, json_file, ensure_ascii=False, indent=4)

print(f"Corpus saved to: {corpus_merged_path}")


In [ ]:
# corpus_en.json
import csv
import json

# Input CSV files
csv_files = [
    '/content/bangla_scraped_words_v2 - 10k_1.csv',
    '/content/bangla_scraped_words_v2 - 10k_2.csv',
    '/content/bangla_scraped_words_v2 - 10k_3.csv',
    '/content/bangla_scraped_words_v2 - 10k_4.csv',
    '/content/bangla_scraped_words_v2 - 10k_5.csv',
    '/content/bangla_scraped_words_v2 - 10k_6.csv',
    '/content/bangla_scraped_words_v2 - 10k_7.csv',
    '/content/bangla_scraped_words_v2 - 10k_8.csv',
    '/content/bangla_scraped_words_v2 - 10k_9.csv',
    '/content/bangla_scraped_words_v2 - 10k_10.csv',
    '/content/bangla_scraped_words_v2 - old_10k_1.csv'
]

unique_words_ordered = {}

for csv_file in csv_files:
    try:
        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row.get('Name?', '').strip().lower() == 'y':
                    continue
                word = row['Corrected'].strip() if row['Corrected'].strip() else row['English'].strip().lower()
                if word and word not in unique_words_ordered:
                    unique_words_ordered[word] = None
    except Exception as e:
        print(f"Failed to process file {csv_file}: {e}")

corpus_en_dict = {str(i): word for i, word in enumerate(unique_words_ordered.keys())}

corpus_en_path = '/content/corpus_en.json'
with open(corpus_en_path, 'w', encoding='utf-8') as json_file:
    json.dump(corpus_en_dict, json_file, ensure_ascii=False, indent=4)

print(f"Saved unique ordered corpus to: {corpus_en_path}")


In [ ]:
# missing bn words in csv
import pandas as pd
import re

def clean_token(token):
    # Remove "৷", "ঃ", and any whitespace characters
    return re.sub(r"[৷ঃ\s]+", "", token)

def tokenize_bangla_text(text):
    if pd.isna(text):
        return []
    tokens = re.findall(r'[\u0980-\u09FF]+', text)
    return [clean_token(token) for token in tokens if clean_token(token)]

sentiment_df = pd.read_csv('/content/Sentiment Analysis Bangla.csv')

all_comment_words = set()
for sentence in sentiment_df['comment'].dropna():
    all_comment_words.update(tokenize_bangla_text(sentence))

csv_files = [
    '/content/bangla_scraped_words_v2 - 10k_1.csv',
    '/content/bangla_scraped_words_v2 - 10k_2.csv',
    '/content/bangla_scraped_words_v2 - 10k_3.csv',
    '/content/bangla_scraped_words_v2 - 10k_4.csv',
    '/content/bangla_scraped_words_v2 - 10k_5.csv',
    '/content/bangla_scraped_words_v2 - 10k_6.csv',
    '/content/bangla_scraped_words_v2 - 10k_7.csv',
    '/content/bangla_scraped_words_v2 - 10k_8.csv',
    '/content/bangla_scraped_words_v2 - 10k_9.csv',
    '/content/bangla_scraped_words_v2 - 10k_10.csv',
    '/content/bangla_scraped_words_v2 - old_10k_1.csv'
]

known_bangla_words = set()
for file in csv_files:
    df = pd.read_csv(file)
    if 'Bangla' in df.columns:
        words = df['Bangla'].dropna().astype(str).apply(lambda w: clean_token(w.strip()))
        known_bangla_words.update(words)

missing_words = sorted(all_comment_words - known_bangla_words)

print("Number of missing words:", len(missing_words))
print("Sample missing words:", missing_words[:20])

pd.DataFrame(missing_words, columns=['Missing_Bangla_Words']).to_csv('missing_bangla_words.csv', index=False)
